In [1]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.random import set_seed

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GATConv
from spektral.transforms import LayerPreprocess

In [2]:
dataset = Citation("cora", normalize_x=True, transforms=[LayerPreprocess(GATConv)])

def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)

weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

Pre-processing node features


C:\Users\Ayush\.conda\envs\gcn\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
channels = 8  # Number of channels in each head of the first GAT layer
n_attn_heads = 8  # Number of attention heads in first GAT layer
dropout = 0.6  # Dropout rate for the features and adjacency matrix
l2_reg = 2.5e-4  # L2 regularization rate
learning_rate = 5e-3  # Learning rate
epochs = 20000  # Number of training epochs
patience = 100  # Patience for early stopping

N = dataset.n_nodes  # Number of nodes in the graph
F = dataset.n_node_features  # Original size of node features
n_out = dataset.n_labels  # Number of classes

x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True)

do_1 = Dropout(dropout)(x_in)
gc_1 = GATConv(
    channels,
    attn_heads=n_attn_heads,
    concat_heads=True,
    dropout_rate=dropout,
    activation="elu",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([do_1, a_in])
do_2 = Dropout(dropout)(gc_1)
gc_2 = GATConv(
    n_out,
    attn_heads=1,
    concat_heads=False,
    dropout_rate=dropout,
    activation="softmax",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([do_2, a_in])

# Build model
model = Model(inputs=[x_in, a_in], outputs=gc_2)
optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1433)]       0           []                               
                                                                                                  
 dropout_2 (Dropout)            (None, 1433)         0           ['input_3[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 2708)]       0           []                               
                                                                                                  
 gat_conv_2 (GATConv)           (None, 64)           91904       ['dropout_2[0][0]',              
                                                                  'input_4[0][0]']          

In [5]:
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/20000
1/1 [==============================] - 2s 2s/step - loss: 1.9498 - acc: 0.1000 - val_loss: 1.9495 - val_acc: 0.1580
Epoch 2/20000
1/1 [==============================] - 0s 76ms/step - loss: 1.9473 - acc: 0.2071 - val_loss: 1.9479 - val_acc: 0.2700
Epoch 3/20000
1/1 [==============================] - 0s 73ms/step - loss: 1.9452 - acc: 0.2929 - val_loss: 1.9463 - val_acc: 0.1920
Epoch 4/20000
1/1 [==============================] - 0s 73ms/step - loss: 1.9425 - acc: 0.3071 - val_loss: 1.9442 - val_acc: 0.4220
Epoch 5/20000
1/1 [==============================] - 0s 78ms/step - loss: 1.9403 - acc: 0.3643 - val_loss: 1.9417 - val_acc: 0.7020
Epoch 6/20000
1/1 [==============================] - 0s 76ms/step - loss: 1.9384 - acc: 0.4643 - val_loss: 1.9393 - val_acc: 0.7360
Epoch 7/20000
1/1 [==============================] - 0s 73ms/step - loss: 1.9340 - acc: 0.5714 - val_loss: 1.9370 - val_acc: 0.7740
Epoch 8/20000
1/1 [==============================] - 0s 82ms/step - loss: 1.93

Epoch 63/20000
1/1 [==============================] - 0s 78ms/step - loss: 1.3804 - acc: 0.8000 - val_loss: 1.5924 - val_acc: 0.7900
Epoch 64/20000
1/1 [==============================] - 0s 73ms/step - loss: 1.3983 - acc: 0.8714 - val_loss: 1.5837 - val_acc: 0.7980
Epoch 65/20000
1/1 [==============================] - 0s 72ms/step - loss: 1.4143 - acc: 0.7500 - val_loss: 1.5765 - val_acc: 0.8000
Epoch 66/20000
1/1 [==============================] - 0s 71ms/step - loss: 1.4629 - acc: 0.7643 - val_loss: 1.5688 - val_acc: 0.8020
Epoch 67/20000
1/1 [==============================] - 0s 70ms/step - loss: 1.4251 - acc: 0.7500 - val_loss: 1.5609 - val_acc: 0.8060
Epoch 68/20000
1/1 [==============================] - 0s 71ms/step - loss: 1.3550 - acc: 0.7571 - val_loss: 1.5520 - val_acc: 0.8020
Epoch 69/20000
1/1 [==============================] - 0s 72ms/step - loss: 1.3223 - acc: 0.8143 - val_loss: 1.5416 - val_acc: 0.8060
Epoch 70/20000
1/1 [==============================] - 0s 71ms/step - 

1/1 [==============================] - 0s 69ms/step - loss: 1.1611 - acc: 0.7929 - val_loss: 1.2550 - val_acc: 0.8060
Epoch 125/20000
1/1 [==============================] - 0s 85ms/step - loss: 1.1743 - acc: 0.7929 - val_loss: 1.2523 - val_acc: 0.8080
Epoch 126/20000
1/1 [==============================] - 0s 70ms/step - loss: 1.0821 - acc: 0.8429 - val_loss: 1.2493 - val_acc: 0.8080
Epoch 127/20000
1/1 [==============================] - 0s 71ms/step - loss: 1.0457 - acc: 0.8286 - val_loss: 1.2459 - val_acc: 0.8060
Epoch 128/20000
1/1 [==============================] - 0s 76ms/step - loss: 1.1229 - acc: 0.7929 - val_loss: 1.2416 - val_acc: 0.8040
Epoch 129/20000
1/1 [==============================] - 0s 75ms/step - loss: 1.0800 - acc: 0.8000 - val_loss: 1.2377 - val_acc: 0.8040
Epoch 130/20000
1/1 [==============================] - 0s 90ms/step - loss: 1.1576 - acc: 0.7714 - val_loss: 1.2317 - val_acc: 0.8020
Epoch 131/20000
1/1 [==============================] - 0s 89ms/step - loss: 1.

1/1 [==============================] - 0s 69ms/step - loss: 0.9998 - acc: 0.8357 - val_loss: 1.1204 - val_acc: 0.8040
Epoch 186/20000
1/1 [==============================] - 0s 72ms/step - loss: 1.0020 - acc: 0.7857 - val_loss: 1.1226 - val_acc: 0.8000
Epoch 187/20000
1/1 [==============================] - 0s 70ms/step - loss: 1.0492 - acc: 0.8143 - val_loss: 1.1260 - val_acc: 0.8000
Epoch 188/20000
1/1 [==============================] - 0s 71ms/step - loss: 1.0600 - acc: 0.8071 - val_loss: 1.1289 - val_acc: 0.8000
Epoch 189/20000
1/1 [==============================] - 0s 74ms/step - loss: 0.9598 - acc: 0.8643 - val_loss: 1.1311 - val_acc: 0.8040
Epoch 190/20000
1/1 [==============================] - 0s 71ms/step - loss: 0.9965 - acc: 0.8000 - val_loss: 1.1329 - val_acc: 0.8020
Epoch 191/20000
1/1 [==============================] - 0s 75ms/step - loss: 1.0440 - acc: 0.7857 - val_loss: 1.1342 - val_acc: 0.8000
Epoch 192/20000
1/1 [==============================] - 0s 81ms/step - loss: 1.

1/1 [==============================] - 0s 75ms/step - loss: 0.9851 - acc: 0.8429 - val_loss: 1.0893 - val_acc: 0.8040
Epoch 247/20000
1/1 [==============================] - 0s 75ms/step - loss: 0.9417 - acc: 0.8214 - val_loss: 1.0912 - val_acc: 0.8040
Epoch 248/20000
1/1 [==============================] - 0s 104ms/step - loss: 0.9387 - acc: 0.8429 - val_loss: 1.0920 - val_acc: 0.8040
Epoch 249/20000
1/1 [==============================] - 0s 72ms/step - loss: 0.9830 - acc: 0.8000 - val_loss: 1.0926 - val_acc: 0.8060
Epoch 250/20000
1/1 [==============================] - 0s 73ms/step - loss: 0.8997 - acc: 0.8500 - val_loss: 1.0918 - val_acc: 0.8060
Epoch 251/20000
1/1 [==============================] - 0s 71ms/step - loss: 0.9058 - acc: 0.8857 - val_loss: 1.0901 - val_acc: 0.8040
Epoch 252/20000
1/1 [==============================] - 0s 72ms/step - loss: 0.9529 - acc: 0.8071 - val_loss: 1.0859 - val_acc: 0.8060
Epoch 253/20000
1/1 [==============================] - 0s 73ms/step - loss: 1

...

1/1 [==============================] - 0s 73ms/step - loss: 0.8820 - acc: 0.8429 - val_loss: 1.0795 - val_acc: 0.8000
Epoch 369/20000
1/1 [==============================] - 0s 74ms/step - loss: 0.9211 - acc: 0.8571 - val_loss: 1.0768 - val_acc: 0.8000
Epoch 370/20000
1/1 [==============================] - 0s 72ms/step - loss: 0.8729 - acc: 0.8500 - val_loss: 1.0719 - val_acc: 0.8000
Epoch 371/20000
1/1 [==============================] - 0s 78ms/step - loss: 0.9154 - acc: 0.8429 - val_loss: 1.0650 - val_acc: 0.8000
Epoch 372/20000
1/1 [==============================] - 0s 75ms/step - loss: 0.9448 - acc: 0.8214 - val_loss: 1.0565 - val_acc: 0.7960
Epoch 373/20000
1/1 [==============================] - 0s 78ms/step - loss: 0.9536 - acc: 0.8500 - val_loss: 1.0499 - val_acc: 0.7980
Epoch 374/20000
1/1 [==============================] - 0s 76ms/step - loss: 0.8439 - acc: 0.8857 - val_loss: 1.0457 - val_acc: 0.8000
Epoch 375/20000
1/1 [==============================] - 0s 86ms/step - loss: 0.

1/1 [==============================] - 0s 78ms/step - loss: 0.9775 - acc: 0.7929 - val_loss: 1.0555 - val_acc: 0.7940
Epoch 430/20000
1/1 [==============================] - 0s 77ms/step - loss: 0.8610 - acc: 0.8643 - val_loss: 1.0563 - val_acc: 0.7940
Epoch 431/20000
1/1 [==============================] - 0s 74ms/step - loss: 0.9287 - acc: 0.8286 - val_loss: 1.0551 - val_acc: 0.7920
Epoch 432/20000
1/1 [==============================] - 0s 77ms/step - loss: 0.8791 - acc: 0.8429 - val_loss: 1.0559 - val_acc: 0.7900
Epoch 433/20000
1/1 [==============================] - 0s 72ms/step - loss: 0.8621 - acc: 0.8714 - val_loss: 1.0565 - val_acc: 0.7900
Epoch 434/20000
1/1 [==============================] - 0s 80ms/step - loss: 0.8956 - acc: 0.8571 - val_loss: 1.0590 - val_acc: 0.7900
Epoch 435/20000
1/1 [==============================] - 0s 79ms/step - loss: 0.9169 - acc: 0.8214 - val_loss: 1.0616 - val_acc: 0.7880
Epoch 436/20000
1/1 [==============================] - 0s 73ms/step - loss: 0.

1/1 [==============================] - 0s 78ms/step - loss: 0.8578 - acc: 0.8643 - val_loss: 1.0319 - val_acc: 0.8020
Epoch 491/20000
1/1 [==============================] - 0s 81ms/step - loss: 0.9119 - acc: 0.8643 - val_loss: 1.0372 - val_acc: 0.8020
Epoch 492/20000
1/1 [==============================] - 0s 90ms/step - loss: 0.8954 - acc: 0.8571 - val_loss: 1.0474 - val_acc: 0.7960
Epoch 493/20000
1/1 [==============================] - 0s 82ms/step - loss: 0.9646 - acc: 0.8571 - val_loss: 1.0583 - val_acc: 0.7980
Epoch 494/20000
1/1 [==============================] - 0s 78ms/step - loss: 0.8763 - acc: 0.8643 - val_loss: 1.0675 - val_acc: 0.7940
Epoch 495/20000
1/1 [==============================] - 0s 77ms/step - loss: 0.8030 - acc: 0.8714 - val_loss: 1.0724 - val_acc: 0.7980
Epoch 496/20000
1/1 [==============================] - 0s 82ms/step - loss: 0.9846 - acc: 0.8214 - val_loss: 1.0764 - val_acc: 0.7980
Epoch 497/20000
1/1 [==============================] - 0s 83ms/step - loss: 0.

1/1 [==============================] - 0s 82ms/step - loss: 0.9005 - acc: 0.8571 - val_loss: 1.0496 - val_acc: 0.7940
Epoch 552/20000
1/1 [==============================] - 0s 85ms/step - loss: 0.8583 - acc: 0.8643 - val_loss: 1.0498 - val_acc: 0.7940
Epoch 553/20000
1/1 [==============================] - 0s 85ms/step - loss: 0.9533 - acc: 0.8643 - val_loss: 1.0468 - val_acc: 0.7960
Epoch 554/20000
1/1 [==============================] - 0s 80ms/step - loss: 0.9199 - acc: 0.8429 - val_loss: 1.0450 - val_acc: 0.7940
Epoch 555/20000
1/1 [==============================] - 0s 86ms/step - loss: 1.0009 - acc: 0.7857 - val_loss: 1.0436 - val_acc: 0.7920
Epoch 556/20000
1/1 [==============================] - 0s 79ms/step - loss: 0.9161 - acc: 0.8429 - val_loss: 1.0444 - val_acc: 0.7900
Epoch 557/20000
1/1 [==============================] - 0s 80ms/step - loss: 0.9515 - acc: 0.8143 - val_loss: 1.0443 - val_acc: 0.7880
Epoch 558/20000
1/1 [==============================] - 0s 84ms/step - loss: 0.

1/1 [==============================] - 0s 75ms/step - loss: 0.8692 - acc: 0.8571 - val_loss: 1.0584 - val_acc: 0.7900
Epoch 613/20000
1/1 [==============================] - 0s 76ms/step - loss: 0.8271 - acc: 0.9000 - val_loss: 1.0591 - val_acc: 0.7880
Epoch 614/20000
1/1 [==============================] - 0s 78ms/step - loss: 0.8599 - acc: 0.8286 - val_loss: 1.0531 - val_acc: 0.7920
Epoch 615/20000
1/1 [==============================] - 0s 75ms/step - loss: 0.9349 - acc: 0.8357 - val_loss: 1.0490 - val_acc: 0.7980
Epoch 616/20000
1/1 [==============================] - 0s 79ms/step - loss: 0.8809 - acc: 0.8571 - val_loss: 1.0454 - val_acc: 0.8020
Epoch 617/20000
1/1 [==============================] - 0s 75ms/step - loss: 0.8121 - acc: 0.8643 - val_loss: 1.0419 - val_acc: 0.8020
Epoch 618/20000
1/1 [==============================] - 0s 76ms/step - loss: 0.9460 - acc: 0.8500 - val_loss: 1.0403 - val_acc: 0.8020
Epoch 619/20000
1/1 [==============================] - 0s 78ms/step - loss: 0.

In [6]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 31ms/step - loss: 0.9855 - acc: 0.8350
Done.
Test loss: 0.9854999780654907
Test accuracy: 0.8349997997283936
